In [31]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import XGBModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Configurações de visualização
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [32]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

df = pd.read_csv("../data/ground_station_clean.csv")

In [33]:
encoders = {
    "cyclic":{
        "past":["month","dayofyear","day","hour","minute"],
        "future":["month","dayofyear","day","hour","minute"],
    },
    "transformer": Scaler()
}

In [34]:
class PatchedPruningCallback(optuna.integration.PyTorchLightningPruningCallback, Callback):
    pass
# Criar diretório para salvar os resultados das iterações
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [41]:
def objective(trial):
    num_target_lags = trial.suggest_int('num_target_lags', 5, 24)
    target_lags_list = [-i for i in range(1, num_target_lags + 1)]
    output_chunk_length = trial.suggest_int('output_chunk_length', 1, 24)
    forecast_horizon = trial.suggest_int('forecast_horizon', 1, 24)
    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600, step=50),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.1),
        'gamma': trial.suggest_float('gamma', 0, 0.3, step=0.05),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 0.5, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 0.5, log=True),
        'random_state': 42, # Para reprodutibilidade
        'n_jobs': -1 # Usar todos os processadores
    } 

    pruner = PatchedPruningCallback(trial, monitor="val_loss")
    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=10, verbose=True)
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [pruner, early_stopper],
    }
    n = int(len(df)*0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nIniciando Trial {trial.number}")
    print(f"Hiperparâmetros: {trial.params}")
    print("\nTreinando o modelo...")
    print(f"Train set: {len(train_fold)} samples") # Usar len() para TimeSeries
    print(f"Validation set: {len(val_fold)} samples") # Usar len() para TimeSeries
    
    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    # Definir work_dir e model_name para consistência
    # force_reset=True irá limpar work_dir/model_name antes de cada treinamento da iteração.
    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/" # Certifique-se que este caminho existe e é gravável
    _model_name = "model_optuna_temp" # Nome do modelo temporário para cada iteração

    # Criar o diretório _work_dir se não existir
    os.makedirs(_work_dir, exist_ok=True)
    model = XGBModel(
        model_name=_model_name,
        work_dir=_work_dir,
        lags=target_lags_list,
        output_chunk_length=1,
        pl_trainer_kwargs=pl_trainer_kwargs,
        add_encoders=encoders,
        **xgb_params
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=True,
    )
    


    forecasts = model.historical_forecasts(
        train_scaled,# Como no script original
        start=0.8, # Início da previsão no conjunto de treinamento
        forecast_horizon=forecast_horizon,
        stride=1, # Gera uma previsão em cada ponto de tempo possível
        retrain=False,
        verbose=True # Pode ser False
    ) 

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(train_scaled).split_after(0.8)[1]
    metrics = {}
    print("Iniciando a verificação das séries temporais para constância...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nMétricas de performance:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_smape_val = smape(s, forecasts_t)
    print(f"O SMAPE desse fold foi {overall_smape_val}")

    trial_dict = {
        "trial_number": trial.number,
        "forecast_horizon": forecast_horizon,
        "overall_smape_val": overall_smape_val,
        "output_chunk_length": output_chunk_length,
        "xgb_params": xgb_params,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Resultados do trial {trial.number} salvos em {json_path}")
    return overall_smape_val if not np.isnan(overall_smape_val) else float("inf")



In [42]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [43]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, callbacks=[print_callback])

[I 2025-05-15 22:08:38,487] A new study created in memory with name: no-name-b92163cf-fded-42c7-b304-5a1dfd20cbd5
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.



Iniciando Trial 0
Hiperparâmetros: {'num_target_lags': 23, 'output_chunk_length': 22, 'forecast_horizon': 20, 'n_estimators': 550, 'max_depth': 8, 'learning_rate': 0.09303278703055912, 'min_child_weight': 4, 'subsample': 0.6, 'colsample_bytree': 0.6, 'gamma': 0.2, 'reg_alpha': 0.00012911256801917276, 'reg_lambda': 0.4115575899676296}

Treinando o modelo...
Train set: 6647 samples
Validation set: 1662 samples
[0]	validation_0-rmse:0.33478


[22:08:38] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "model_name", "pl_trainer_kwargs", "work_dir" } are not used.



[1]	validation_0-rmse:0.31178
[2]	validation_0-rmse:0.29134
[3]	validation_0-rmse:0.27238
[4]	validation_0-rmse:0.25610
[5]	validation_0-rmse:0.24202
[6]	validation_0-rmse:0.23026
[7]	validation_0-rmse:0.21926
[8]	validation_0-rmse:0.21015
[9]	validation_0-rmse:0.20314
[10]	validation_0-rmse:0.19619
[11]	validation_0-rmse:0.18968
[12]	validation_0-rmse:0.18473
[13]	validation_0-rmse:0.18020
[14]	validation_0-rmse:0.17643
[15]	validation_0-rmse:0.17377
[16]	validation_0-rmse:0.17095
[17]	validation_0-rmse:0.16844
[18]	validation_0-rmse:0.16654
[19]	validation_0-rmse:0.16468
[20]	validation_0-rmse:0.16314
[21]	validation_0-rmse:0.16177
[22]	validation_0-rmse:0.16085
[23]	validation_0-rmse:0.16016
[24]	validation_0-rmse:0.15956
[25]	validation_0-rmse:0.15911
[26]	validation_0-rmse:0.15859
[27]	validation_0-rmse:0.15829
[28]	validation_0-rmse:0.15783
[29]	validation_0-rmse:0.15766
[30]	validation_0-rmse:0.15749
[31]	validation_0-rmse:0.15732
[32]	validation_0-rmse:0.15709
[33]	validation_0

`enable_optimization=True` is ignored because `forecast_horizon > model.output_chunk_length`. To hide this warning, set `show_warnings=False` or `enable_optimization=False`.


...

[0]	validation_0-rmse:0.35244
[1]	validation_0-rmse:0.34341
[2]	validation_0-rmse:0.33488
[3]	validation_0-rmse:0.32653
[4]	validation_0-rmse:0.31860
[5]	validation_0-rmse:0.31101
[6]	validation_0-rmse:0.30343
[7]	validation_0-rmse:0.29656
[8]	validation_0-rmse:0.28996
[9]	validation_0-rmse:0.28335
[10]	validation_0-rmse:0.27733
[11]	validation_0-rmse:0.27150
[12]	validation_0-rmse:0.26584
[13]	validation_0-rmse:0.26069
[14]	validation_0-rmse:0.25571
[15]	validation_0-rmse:0.25037
[16]	validation_0-rmse:0.24548
[17]	validation_0-rmse:0.24102
[18]	validation_0-rmse:0.23672
[19]	validation_0-rmse:0.23270
[20]	validation_0-rmse:0.22873
[21]	validation_0-rmse:0.22474
[22]	validation_0-rmse:0.22121
[23]	validation_0-rmse:0.21746
[24]	validation_0-rmse:0.21402
[25]	validation_0-rmse:0.21103
[26]	validation_0-rmse:0.20792
[27]	validation_0-rmse:0.20507
[28]	validation_0-rmse:0.20241
[29]	validation_0-rmse:0.19983
[30]	validation_0-rmse:0.19731
[31]	validation_0-rmse:0.19489
[32]	validation_0-

`enable_optimization=True` is ignored because `forecast_horizon > model.output_chunk_length`. To hide this warning, set `show_warnings=False` or `enable_optimization=False`.


historical forecasts:   0%|          | 0/1324 [00:00<?, ?it/s]

[I 2025-05-15 22:20:08,020] Trial 49 finished with value: 42.55418537907892 and parameters: {'num_target_lags': 14, 'output_chunk_length': 8, 'forecast_horizon': 8, 'n_estimators': 350, 'max_depth': 7, 'learning_rate': 0.032138420704449576, 'min_child_weight': 3, 'subsample': 1.0, 'colsample_bytree': 0.6, 'gamma': 0.0, 'reg_alpha': 0.002395764447978629, 'reg_lambda': 0.47045953429360754}. Best is trial 48 with value: 41.92694797342445.


Iniciando a verificação das séries temporais para constância...

Métricas de performance:
                                    MSE        RMSE  ...        R2       SMAPE
Temperature                    0.755106    0.868969  ...  0.777670    8.262456
Precipitation_accumulated      0.020725    0.143961  ...  0.978186    0.023165
Humidity                       4.825753    2.196760  ...  0.925838    1.325901
Wind_Speed_kmh                 0.026332    0.162271  ...  0.922947  146.870101
Soil_Moisture              19244.341014  138.723974  ...  0.974096    0.582342
Soil_Temperature               0.774074    0.879815  ...  0.861808   13.024286
Wind_Dir_Sin                   0.045566    0.213461  ...  0.717437  146.345951
Wind_Dir_Cos                   0.038235    0.195538  ...  0.867267   23.999282

[8 rows x 5 columns]
O SMAPE desse fold foi 42.55418537907892
Resultados do trial 49 salvos em optuna_iteration_metrics/trial_49.json
Current value: 42.55418537907892, Current params: {'num_target_l

In [44]:
print(f"Melhor Valor (Mínimo): {study.best_value}")
print(f"Melhores Parâmetros: {study.best_params}")

Melhor Valor (Mínimo): 41.92694797342445
Melhores Parâmetros: {'num_target_lags': 14, 'output_chunk_length': 8, 'forecast_horizon': 8, 'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.03200570242677771, 'min_child_weight': 3, 'subsample': 1.0, 'colsample_bytree': 0.6, 'gamma': 0.0, 'reg_alpha': 0.004067293711184323, 'reg_lambda': 0.3984658853981022}


In [45]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Melhores resultados salvos em {json_path}")

Melhores resultados salvos em optuna_iteration_metrics/best_trial.json
